In [1]:
from shell.datasets.datasets import get_dataset
import numpy as np
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
from shell.utils.utils import seed_everything
import torch
import subprocess
import torch.nn as nn
import torch
import os
from omegaconf import DictConfig
from shell.datasets.datasets import get_dataset
from shell.utils.utils import seed_everything
from pprint import pprint
from shell.fleet.network import TopologyGenerator
from shell.models.cnn_soft_lifelong_dynamic import CNNSoftLLDynamic
from shell.models.cnn import CNN
from shell.models.mlp import MLP
from shell.models.mlp_soft_lifelong_dynamic import MLPSoftLLDynamic
from shell.learners.er_dynamic import CompositionalDynamicER
from shell.learners.er_nocomponents import NoComponentsER
from shell.utils.experiment_utils import eval_net
from shell.utils.experiment_utils import setup_experiment
from sklearn.manifold import TSNE
import logging
import seaborn as sns
import torchvision.transforms as transforms
logging.basicConfig(level=logging.INFO)


seed_everything(0)

In [2]:
data_cfg = {
    "dataset_name": "cifar100",
    "num_tasks": 4,
    "num_train_per_task": 256,
    'remap_labels': True,
    "with_replacement": True,
}


dataset = get_dataset(**data_cfg)

INFO:root:Class sequence: [97 66 33 19 34 60 63  3 66 34 73 98  4 63 53 29 32 93 16 40]
INFO:root:task 0 :(256, 3, 32, 32)
INFO:root:task 1 :(256, 3, 32, 32)
INFO:root:task 2 :(256, 3, 32, 32)
INFO:root:task 3 :(256, 3, 32, 32)


In [3]:
"""
Define NetCls and AgentCls
"""

net_cfg = {
    'depth': 4,
    'channels': 50,
    'num_init_tasks': 4,
    'i_size': 32,
    'num_classes': 5,
    'num_tasks': 4,
    'padding':1,
}

agent_cfg = {
    'memory_size': 128,
    # 'use_contrastive': True,
    'use_contrastive': False,
    'dataset_name': data_cfg['dataset_name'],
}
net = CNN(**net_cfg)
agent = NoComponentsER(net, **agent_cfg)

In [4]:
for task_id in range(4):
    trainloader = torch.utils.data.DataLoader(dataset.trainset[task_id],
                                                batch_size=64,
                                                shuffle=True,
                                                num_workers=0,
                                                pin_memory=True,
                                                )
    testloaders = {task: torch.utils.data.DataLoader(testset,
                                                            batch_size=128,
                                                            shuffle=False,
                                                            num_workers=0,
                                                            pin_memory=True,
                                                            ) for task, testset in enumerate(dataset.testset[:(task_id+1)])}
    agent.train(trainloader, task_id, num_epochs=100, testloaders=testloaders)
                

INFO:root:epochs: 0, training task: 0
INFO:root:	task: 0	loss: 1.610	acc: 0.220
INFO:root:	task: avg	loss: 1.610	acc: 0.220
INFO:root:epochs: 0, training task: 0
INFO:root:	task: 0	loss: 1.610	acc: 0.220
INFO:root:	task: avg	loss: 1.610	acc: 0.220
INFO:root:epochs: 0, training task: 1
INFO:root:	task: 0	loss: 1.610	acc: 0.220
INFO:root:	task: 1	loss: 1.610	acc: 0.200
INFO:root:	task: avg	loss: 1.610	acc: 0.210
INFO:root:epochs: 0, training task: 1
INFO:root:	task: 0	loss: 1.610	acc: 0.220
INFO:root:	task: 1	loss: 1.610	acc: 0.200
INFO:root:	task: avg	loss: 1.610	acc: 0.210
INFO:root:epochs: 0, training task: 2
INFO:root:	task: 0	loss: 1.610	acc: 0.220
INFO:root:	task: 1	loss: 1.610	acc: 0.200
INFO:root:	task: 2	loss: 1.610	acc: 0.200
INFO:root:	task: avg	loss: 1.610	acc: 0.207
INFO:root:epochs: 0, training task: 2
INFO:root:	task: 0	loss: 1.610	acc: 0.220
INFO:root:	task: 1	loss: 1.610	acc: 0.200
INFO:root:	task: 2	loss: 1.610	acc: 0.200
INFO:root:	task: avg	loss: 1.610	acc: 0.207
INFO